In [14]:
import pandas as pd
import string
import nltk

In [15]:
from nltk.corpus import stopwords

In [16]:
st = stopwords.words('english')
st.append("and")

In [17]:
products = pd.read_csv('/kaggle/input/amazon-baby-dataset/reviews_Baby_5_final_dataset.csv')

In [18]:
products.head()

,reviewerID,asin,reviewerName,helpful,helpful_num,helpful_den,reviewText,overall,summary,unixReviewTime,reviewTime,exclamationcount,questioncount,charcount,wordcount,capcount,avgrating,diffrating,ishelpful
0,A3NMPMELAZC8ZY,097293751X,Jakell,"[3, 3]",3,3,This book is perfect! I'm a first time new mo...,5,Great for newborns,1359244800,"01 27, 2013",1,0,250,46,0,4.0,1.0,1
1,A3O4ATU0ENBKTU,097293751X,MAPN,"[1, 1]",1,1,I use this so that our babysitter (grandma) ca...,5,Compact and Easy way to record the milestones,1361836800,"02 26, 2013",0,0,734,148,0,4.0,1.0,1
2,A2SYNL4YX73KNY,097293751X,"R. Davidson ""Jrdpa""","[2, 2]",2,2,"I like this log, but think it would work bette...",3,Needs clearer AM & PM,1369008000,"05 20, 2013",0,0,288,59,2,4.0,1.0,1
3,A2Q2A6JKY95RTP,097293751X,R. Garrelts,"[2, 2]",2,2,My wife and I have a six month old baby boy an...,3,Expensive and Somewhat Limited Format,1381968000,"10 17, 2013",0,0,2959,505,9,4.0,1.0,1
4,A21I33AWNOWMK8,9729375011,EmilyS,"[1, 2]",1,2,I have used this book since my son was born. ...,5,Great product!,1364256000,"03 26, 2013",0,0,595,117,0,4.5,0.5,0


In [19]:
def remove_punctuation(message):
    test_punc_removed = [char for char in str(message) if char not in string.punctuation]
    test_punc_removed = ''.join(test_punc_removed)
    test_punc_st_removed = []
    for char in test_punc_removed.split():
        if char.lower() not in st:
            test_punc_st_removed.append(char)
    test_punc_st_removed = " ".join(test_punc_st_removed)  
    return test_punc_st_removed

In [20]:
products['review_clean'] = products['reviewText'].apply(remove_punctuation)

In [21]:
products = products.fillna({'review':''})

In [23]:
products = products[products['overall'] != 3] #remove neutrals

In [25]:
products['sentiment'] = products['overall'].apply(lambda rating : +1 if rating > 3 else -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X = products.drop('sentiment', axis = 1)
y = products['sentiment']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') # Use this token pattern to keep single-letter words

X_train_matrix = vectorizer.fit_transform(X_train['review_clean'])
X_test_matrix = vectorizer.transform(X_test['review_clean'])

In [30]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter = 10000)
logreg.fit(X_train_matrix, y_train)

LogisticRegression(max_iter=10000)

In [31]:
len(logreg.coef_[logreg.coef_>=0])

63725

In [32]:
sample_test_data = X_test[15:25]
print(sample_test_data) 

           reviewerID        asin       reviewerName    helpful  helpful_num  \
9535   A117O44ZF2WD7D  B000GK5XY2              jojo22    [1, 1]            1   
37876  A1RQQ2HWUGEUWA  B004GCJMGG         EB MamaBear    [1, 1]            1   
19190  A31SYTULVH01CL  B001DNJBDK   Pink & Blue Momma    [3, 4]            3   
11137  A1FGTPKGE9T0KM  B000JVTT98             busybee    [0, 1]            0   
27518  A2DWYIVRS48XK8  B002US8D5Y      Nancy Staltman  [13, 19]           13   
18355  A2LDW7RZSW56UO  B001BZC4T8          goal13keep    [2, 2]            2   
6118   A18HZCWRMROOOO  B00078ANK2  PrincessMom "Bron"    [4, 5]            4   
23358  A3QPRPO8MWYO7I  B0028H3ACS    GIA's Mom "Judi"    [1, 2]            1   
32933   AL1AQWGA146ZA  B003LQU0GO     DH "truthinall"    [0, 2]            0   
44747  A2AXKFOLRXZDMC  B005WPPRJM            Allygirl    [0, 1]            0   

       helpful_den                                         reviewText  \
9535             1  I just started using these

In [33]:
sample_test_data['review_clean']

9535     started using 5 month old daughters One extrem...
37876    needed booster 4 year old needs little height ...
19190    crib convertible top front solid wood 10 inche...
11137    bought product came bottle bubble solution min...
27518    1 star simply description product ADEN adenana...
18355    youre looking gigantic diaper bag look biggest...
6118     admit extremly skeptical mother actually bough...
23358    first bought replace 12 pacifiers gone missing...
32933    hate rate 1 star could assemble working two ho...
44747    Bought seeing target big splurge pregnancy som...
Name: review_clean, dtype: object

In [34]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = logreg.decision_function(sample_test_matrix)
print(scores)

[ 3.41679946  7.03208594  7.85793424 -1.06845905 -2.84812596 11.0626689
 11.6550442  36.41213758 -7.30337601  2.74789199]


In [35]:
y_pred = logreg.predict(X_test_matrix)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
ac = accuracy_score(y_test, y_pred)

In [38]:
ac

0.8983633057183269

In [39]:
probs = 1/(1+2.71828**(-1*scores))

In [40]:
probs

array([9.68225384e-01, 9.99117687e-01, 9.99613476e-01, 2.55696377e-01,
       5.47783686e-02, 9.99984313e-01, 9.99991325e-01, 1.00000000e+00,
       6.72812321e-04, 9.39794083e-01])